In [12]:
class Student:

    def __init__ (self, name, age):
        self.name = name
        self.age = age
        # self.home = home


    def bark(self, to):
        self.to = to
        print(f"{self.name} says Woof to {self.to}")

    # def __str__(self):
    #     return f"Student(name={self.name}, age={self.age})"
    
student = Student("John", 20)
student.bark("everyone")
print(student)  # Output: Student(name=John, age=20)



John says Woof to everyone


In [ ]:
import torch
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__() # Initialize nn.Module
        self.fc = nn.Linear(10, 5) # Linear layer with learnable weights

    def forward(self, x, y):
        return self.fc(x)
        
    
    def __str__(self):
        return f"SimpleNet with {len(list(self.parameters()))} parameters"
    
    def named_parameters(self):
        return self.fc.named_parameters()
    
    def parameters(self):
        return self.fc.parameters()
    
    def to(self, device):
        self.fc.to(device)

    def cpu(self):
        self.fc.cpu()

    
    
net = SimpleNet()
x = torch.randn(1, 10)
out = net(x) # Calls forward

print(out.shape) # Output: torch.Size([1, 5])

torch.Size([1, 5])


In [ ]:
net()

TypeError: linear(): argument 'input' (position 1) must be Tensor, not str

In [39]:
net.forward(x)  # Explicitly calling forward method

tensor([[-1.0020,  0.5180, -0.1158, -1.3222,  0.2072]],
       grad_fn=<AddmmBackward0>)

In [34]:
x

tensor([[ 0.6731, -1.0076, -0.0105, -1.1323, -0.7731, -1.0466, -0.4066, -1.5147,
         -0.0939,  0.7078]])

In [33]:
out

tensor([[-0.1254,  0.3057, -0.6936,  0.0105, -0.3183]],
       grad_fn=<AddmmBackward0>)

In [32]:
print(net)

SimpleNet(
  (fc): Linear(in_features=10, out_features=5, bias=True)
)


In [20]:
# Let's see what nn.Linear(10, 5) creates
linear_layer = nn.Linear(10, 5)
print("=== Linear Layer Structure ===")
print(f"Weight shape: {linear_layer.weight.shape}")
print(f"Bias shape: {linear_layer.bias.shape}")
print(f"Weight requires_grad: {linear_layer.weight.requires_grad}")

# When you do self.fc = nn.Linear(10, 5), nn.Module automatically detects this
net = SimpleNet()
print("\n=== Auto-registered parameters ===")
for name, param in net.named_parameters():
    print(f"{name}: shape={param.shape}, requires_grad={param.requires_grad}")

=== Linear Layer Structure ===
Weight shape: torch.Size([5, 10])
Bias shape: torch.Size([5])
Weight requires_grad: True

=== Auto-registered parameters ===
fc.weight: shape=torch.Size([5, 10]), requires_grad=True
fc.bias: shape=torch.Size([5]), requires_grad=True


In [ ]:
# When you do net(x), this is what actually happens:
x = torch.randn(1, 10)

# These are equivalent:
out1 = net(x)           # Calls net.__call__(x)
out2 = net.forward(x)   # Direct forward call

print(f"Are outputs equal? {torch.equal(out1, out2)}")

# Let's see nn.Module's __call__ method (simplified version):
"""
def __call__(self, *input, **kwargs):
    # This is inside nn.Module class
    result = self.forward(*input, **kwargs)
    return result
"""

Are outputs equal? True


'\ndef __call__(self, *input, **kwargs):\n    # This is inside nn.Module class\n    result = self.forward(*input, **kwargs)\n    return result\n'